# 📊Data Cleaning

In [9]:
import pandas as pd
import numpy as np

In [10]:

df_appnt = pd.read_csv('C:/Users/Pranav/Desktop/Portfolio Projects/Hospital Patient Flow/Data raw/appointments.csv')
df_dept = pd.read_csv('C:/Users/Pranav/Desktop/Portfolio Projects/Hospital Patient Flow/Data raw/departments.csv')
df_pat = pd.read_csv('C:/Users/Pranav/Desktop/Portfolio Projects/Hospital Patient Flow/Data raw/patients.csv') 
df_clinic = pd.read_csv('C:/Users/Pranav/Desktop/Portfolio Projects/Hospital Patient Flow/Data raw/clinicians.csv')

# Depatments

## Action: Domain Integrity Fix (Misspelling: Department name)

In [6]:
# Correct the spelling error
df_dept['department_name'] = df_dept['department_name'].replace('Cardiologgy', 'Cardiology')
print("# Corrected 'Cardiologgy' to 'Cardiology'.")

# Validation
print("\n# Validation: Department name")
display(df_dept['department_name'])

# Corrected 'Cardiologgy' to 'Cardiology'.

# Validation: Department name


0    General Practice
1          Cardiology
2          Pediatrics
3         Orthopedics
4         Dermatology
5           Emergency
6                 ICU
7             Surgery
8           Neurology
Name: department_name, dtype: object

# Patients

## 1. Action: Convert date columns to datetime objects.

In [11]:
# Convert date columns to datetime objects

pat_time_cols = ['date_of_birth', 'registration_date']
df_pat[pat_time_cols] = df_pat[pat_time_cols].apply(pd.to_datetime, errors='coerce')

print("# Date columns successfully converted to datetime objects.")

# Validation
print("\n# Validation: Table Info After Time Conversion:")
df_pat.info(memory_usage='deep')

# Date columns successfully converted to datetime objects.

# Validation: Table Info After Time Conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   patient_id          25000 non-null  int64         
 1   date_of_birth       25000 non-null  datetime64[ns]
 2   gender              25000 non-null  object        
 3   nationality         25000 non-null  object        
 4   insurance_provider  25000 non-null  object        
 5   registration_date   25000 non-null  datetime64[ns]
 6   patient_name        25000 non-null  object        
 7   patient_email       25000 non-null  object        
 8   patient_phone       25000 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 9.2 MB


## 2. Action: Categorical Standardization (Gender)

In [12]:
# Define the mapping dictionary
gender_mapping = {
    'M': 'Male',
    'FEMALE': 'Female',
   # 'F': 'Female'  # Added F for robustness, in case it exists in the raw data
}

# Apply the mapping to the gender column
df_pat['gender'] = df_pat['gender'].replace(gender_mapping)

# Standardize casing
df_pat['gender'] = df_pat['gender'].str.title()

print("# Gender standardization complete: 'M' and 'FEMALE' mapped to standard casing.")

# Validation
print("\n# Validation: Gender Values")
display(df_pat['gender'].value_counts(dropna=False))

# Gender standardization complete: 'M' and 'FEMALE' mapped to standard casing.

# Validation: Gender Values


gender
Female    12502
Male      12498
Name: count, dtype: int64

## 3. Action: Fix Business Logic Violation (Registration Before Birth)

In [13]:
# 1. Records violating the temporal rule: registration must be >= birth
temporal_violation_mask = (
    df_pat['registration_date'].notna() &
    df_pat['date_of_birth'].notna() &
    (df_pat['registration_date'] < df_pat['date_of_birth'])
)

# 2. Flag the violating records for easy isolation in analysis
df_pat['date_logic_error_flag'] = temporal_violation_mask

# 3. Action: Invalidate the unreliable date_of_birth by setting it to NaT
# Note: We choose to invalidate DOB because the registration date (a system timestamp) is often considered more reliable.
df_pat.loc[temporal_violation_mask, 'date_of_birth'] = pd.NaT

violation_count = temporal_violation_mask.sum()
print(f"# Temporal integrity fixed: {violation_count} records flagged and conflicting date_of_birth invalidated (set to NaT).")

# Validation
print("\n# Validation: Date Logic Check (Registration Before Birth)")
date_logic_violations = df_pat[
    df_pat['registration_date'] < df_pat['date_of_birth']
]
print(f"Records where Registration Date < Date of Birth: {len(date_logic_violations)}")

# Temporal integrity fixed: 673 records flagged and conflicting date_of_birth invalidated (set to NaT).

# Validation: Date Logic Check (Registration Before Birth)
Records where Registration Date < Date of Birth: 0


In [14]:
# Post cleaning patients table:

display(df_pat)

,patient_id,date_of_birth,gender,nationality,insurance_provider,registration_date,patient_name,patient_email,patient_phone,date_logic_error_flag
0,10001,1993-02-20,Male,Pakistan,MetLife,2021-03-07,Mary Lopez,kevin42@example.org,+1-538-801-0428,False
1,10002,1937-03-24,Female,UAE,Thiqa,2025-03-16,Claire Vance,jenniferolson@example.com,001-893-656-5741,False
2,10003,1959-12-24,Male,India,Self-Pay,2024-08-23,William Lee,rodriguezrebecca@example.org,+1-853-266-2351x454,False
3,10004,1955-04-07,Male,Egypt,Daman,2024-12-06,Crystal Owens,ysanders@example.com,(576)366-2410x746,False
4,10005,2001-12-20,Male,India,AXA,2022-10-24,Sydney Fisher,gillespiebrittany@example.com,809.363.4398x183,False
...,...,...,...,...,...,...,...,...,...,...
24995,34996,2005-08-27,Female,Philippines,Daman,2021-07-15,Daniel Walls,vrivera@example.com,554-990-0692x543,False
24996,34997,1940-05-10,Female,India,Oman Insurance,2022-01-31,Dr. Mason Russell,thompsonjanice@example.org,527-766-4133x783,False
24997,34998,1988-06-28,Male,India,AXA,2023-05-05,Ashley Lopez,hansonjudy@example.org,001-547-230-2071x05253,False
24998,34999,2006-09-24,Female,Philippines,Self-Pay,2022-11-29,Andrew Barnes,kevin15@example.org,6669596093,False


# Clinicians

## Action: Imputation for Invalid Department ID (Katie Pierce)

**Finding:** Clinician ID 20010 is linked to invalid department_id 999.

**Strategy:** Impute to Department ID 1 based on the cross-reference check (GP designation $\approx$ General Practice department) to maximize data accuracy and linkage.

In [11]:
#Target and invalid IDs
TARGET_DEPT_ID = 1      # General Practice, based on GP designation
INVALID_DEPT_ID = 999   # The invalid ID found during profiling
CLINICIAN_ID = 20010    # Katie Pierce

# Impute the correct department ID for the specific clinician
df_clinic.loc[
    (df_clinic['clinician_id'] == CLINICIAN_ID) & 
    (df_clinic['department_id'] == INVALID_DEPT_ID),
    'department_id'
] = TARGET_DEPT_ID


print(f"# Clinician FK Error Resolved: Clinician {CLINICIAN_ID} re-assigned from ID {INVALID_DEPT_ID} to TARGET ID {TARGET_DEPT_ID} (General Practice).")


# Unique department_id values from the departments master table (df_dept)
valid_department_ids = df_dept['department_id'].astype(str).unique()

# Validation
# Check if the department_id in df_clinic IS NOT IN the list of valid_department_ids.
orphaned_clinicians = df_clinic[
    df_clinic['department_id'].notna() &
    ~df_clinic['department_id'].astype(str).isin(valid_department_ids)
]
print("\n# Validation: Orphaned records in clinician")
print(f"# Clinicians assigned to an Invalid Department ID: {len(orphaned_clinicians)}")

# Clinician FK Error Resolved: Clinician 20010 re-assigned from ID 999 to TARGET ID 1 (General Practice).

# Validation: Orphaned records in clinician
# Clinicians assigned to an Invalid Department ID: 0


In [12]:
# Post cleaning cliniacians table:

display(df_clinic)

,clinician_id,clinician_name,department_id,designation,years_experience
0,20001,Mary Lopez,3,GP,29
1,20002,Claire Vance,6,GP,9
2,20003,William Lee,9,Specialist,10
3,20004,Crystal Owens,6,Specialist,16
4,20005,Sydney Fisher,4,Specialist,11
...,...,...,...,...,...
295,20296,Kimberly Ayers,6,Consultant,16
296,20297,James Manning,6,GP,6
297,20298,Brittney Dudley,1,Consultant,1
298,20299,Teresa Taylor MD,5,Specialist,5


# Appointments

## 1. Action: Convert all timestamp columns to datetime objects.

In [17]:
# Convert time columns data type, object to datetime
appnt_time_cols = ['appointment_datetime', 'check_in_time', 'consultation_start_time',
             'consultation_end_time', 'check_out_time']

df_appnt[appnt_time_cols] = df_appnt[appnt_time_cols].apply(pd.to_datetime, errors='coerce')

print("# Date columns successfully converted to datetime objects.")

# Validation
print("\n# Validation: Table Info After Time Conversion:")
df_appnt.info(memory_usage='deep')

# Date columns successfully converted to datetime objects.

# Validation: Table Info After Time Conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   appointment_id           120000 non-null  int64         
 1   patient_id               120000 non-null  int64         
 2   clinician_id             120000 non-null  int64         
 3   appointment_type         120000 non-null  object        
 4   appointment_datetime     120000 non-null  datetime64[ns]
 5   check_in_time            90005 non-null   datetime64[ns]
 6   consultation_start_time  90005 non-null   datetime64[ns]
 7   consultation_end_time    90005 non-null   datetime64[ns]
 8   check_out_time           90005 non-null   datetime64[ns]
 9   appointment_status       120000 non-null  object        
 10  no_show_flag             120000 

## 2. Action: Fix Business Logic Conflict (Status vs. Flag)
Finding: appointment_id = 500015 has Status = 'Completed' but no_show_flag = TRUE.

In [18]:
# Conflicting record ID
conflict_id = 500015

# Set the flag to FALSE since it is a completed status
df_appnt.loc[
    (df_appnt['appointment_id'] == conflict_id) & (df_appnt['appointment_status'] == 'Completed'),
    'no_show_flag'
] = False

print(f"Corrected Business Logic Conflict (ID {conflict_id}): Flag set to FALSE.")

# Validation
conflicting_records = df_appnt[
    (df_appnt['no_show_flag'] == True) &
    (df_appnt['appointment_status'] != 'No-show')
].copy()

conflict_count = len(conflicting_records)
print("\n# Validation: Status vs. Flag")
print(f"\nConflicting records Found: {conflict_count} record(s)")

Corrected Business Logic Conflict (ID 500015): Flag set to FALSE.

# Validation: Status vs. Flag

Conflicting records Found: 0 record(s)


## 3. Action: Fix Categorical Standardization (Misspelling)
Finding: The appointment_status column contains the misspelled value "Cancleld".

In [19]:
# Replace the misspelled value with the standardized value
df_appnt['appointment_status'] = df_appnt['appointment_status'].replace(
    'Cancleld', 
    'Cancelled'
)

print("Categorical Standardization: 'Cancleld' corrected to 'Cancelled'.")

# Validation
print("\n# Validation: Appointment status values")
display(df_appnt['appointment_status'].value_counts(dropna=False))

Categorical Standardization: 'Cancleld' corrected to 'Cancelled'.

# Validation: Appointment status values


appointment_status
Completed    90005
Cancelled    18098
No-show      11897
Name: count, dtype: int64

## 4.  Action: Resolve Foreign Key Integrity Failures (Ground Truth Imputation)

### Based on the below checks, finalized this cleaning approach 

**Finding:** 393 orphaned department_id records in appointments.

**Action:** Impute all invalid IDs to Department ID 1 (General Practice), aligning with the $\text{GP}$ designation found for the linked clinician (Katie Pierce).

In [20]:
# Check1: Current Volume of each Department(Appointments records in each department except department 999(which has 393 records))

# Target department: 9

print(df_appnt[df_appnt['department_id'] == 9].shape[0])

print(df_appnt[df_appnt['department_id'] == 8].shape[0])

print(df_appnt[df_appnt['department_id'] == 7].shape[0])

print(df_appnt[df_appnt['department_id'] == 6].shape[0])

print(df_appnt[df_appnt['department_id'] == 5].shape[0])

print(df_appnt[df_appnt['department_id'] == 4].shape[0])

print(df_appnt[df_appnt['department_id'] == 3].shape[0])

print(df_appnt[df_appnt['department_id'] == 2].shape[0])

print(df_appnt[df_appnt['department_id'] == 1].shape[0])

11728
11883
11653
14792
11931
16403
12020
17494
11703


In [21]:
# Check2: the designation columns in the df_clinic table for Katie Pierce.   >> Showing: GP (general practice)

target_row_by_dept = df_clinic[df_clinic['department_id'] == 999]

print("\n--- Row Filtered by Invalid Department ID 999 ---")
display(target_row_by_dept)


--- Row Filtered by Invalid Department ID 999 ---


,clinician_id,clinician_name,department_id,designation,years_experience


### Decision Confirmation: Imputation to Department ID 1

The following metrics confirm the analytical viability of imputing all orphaned records (source: **Invalid Department ID 999**) to the confirmed target, **Department ID 1 (General Practice)**.

| Metric | Value | Justification |
| :--- | :--- | :--- |
| **Source of Invalid Data** | **Department ID 999** | The orphaned Foreign Key being corrected. |
| **Target Department** | **ID 1 (General Practice)** | Based on cross-referenced evidence (Clinician GP designation $\rightarrow$ General Practice department). |
| **Current Volume (Dept 1)** | 11,703 appointments | Volume before correction. |
| **Records to Impute** | 393 appointments | Orphaned appointments recovered. |
| **New Volume (After Imputation)** | 12,096 appointments | Calculation: $11,703 + 393 = 12,096$. |
| **Percentage Increase (Skew)** | $3.36\%$ | Calculation: $\frac{393}{11,703} \times 100$. |



**Conclusion:** 

Low Skew: The resulting $3.36\%$ increase in volume for Department ID 1 is well within acceptable analytical tolerance (typically $5\%$ or less). This is not an artificial skew; it is a data recovery effort. 

Maximized Accuracy (The Key Point): This is not arbitrary imputation. This strategy is anchored on triangulated evidence, which is the best form of imputation:

Evidence 1 (Direct): Department Name matches Designation(GP) ($\text{Department 1} \approx \text{General Practice}$).

Evidence 2 (High Confidence): Fixing the FK error for a $\text{GP}$ must lead to the $\text{General Practice}$ department.

Evidence 3 (Assumption Check): It is logical to assume the $393$ associated appointments belong to the same category as the only clinician linked to the error.(clinician_id = 20010, clinician_name: Katie Pierce)

In [22]:
# The target Department ID
TARGET_DEPT_ID = 1

# Get the list of all VALID department IDs
valid_dept_ids = df_dept['department_id'].unique()

# Records where department_id is NOT a valid ID in appointments
orphaned_records_mask = ~df_appnt['department_id'].isin(valid_dept_ids)

# Action: Impute the TARGET_DEPT_ID (1) for all orphaned records
df_appnt.loc[orphaned_records_mask, 'department_id'] = TARGET_DEPT_ID

imputed_count = orphaned_records_mask.sum()
print(f"Resolved Foreign Key Integrity: {imputed_count} orphaned records imputed to Department ID {TARGET_DEPT_ID} (General Practice).")


# Validation
print("\n# Validation: FK integrity issues in appointments")
orphaned_count = orphaned_records_mask.sum()
print(f" Remaining orphaned count: {orphaned_count}")

Resolved Foreign Key Integrity: 393 orphaned records imputed to Department ID 1 (General Practice).

# Validation: FK integrity issues in appointments
 Remaining orphaned count: 393


In [24]:
# Post cleaning Appointments table:

display(df_appnt)

,appointment_id,patient_id,clinician_id,appointment_type,appointment_datetime,check_in_time,consultation_start_time,consultation_end_time,check_out_time,appointment_status,no_show_flag,room_or_area,department_id,encounter_type
0,500001,30626,20162,Emergency,2025-04-20 08:35:51.381719,2025-04-20 08:40:51.381719,2025-04-20 08:57:51.381719,2025-04-20 09:08:51.381719,2025-04-20 09:21:51.381719,Completed,False,Room-225,1,OPD
1,500002,31635,20296,Emergency,2025-02-08 18:08:57.923913,2025-02-08 18:00:57.923913,2025-02-08 18:18:57.923913,2025-02-08 18:34:57.923913,2025-02-08 18:41:57.923913,Completed,False,Room-402,6,ED
2,500003,21618,20115,Procedure,2025-07-15 21:41:44.657416,2025-07-15 21:39:44.657416,2025-07-15 21:51:44.657416,2025-07-15 22:02:44.657416,2025-07-15 22:09:44.657416,Completed,False,Room-211,8,IPD
3,500004,28946,20297,Emergency,2024-12-15 08:01:53.339738,2024-12-15 07:53:53.339738,2024-12-15 08:35:53.339738,2024-12-15 08:46:53.339738,2024-12-15 08:57:53.339738,Completed,False,Room-466,6,ED
4,500005,23377,20009,Follow-up,2025-09-18 10:33:01.055795,NaT,NaT,NaT,NaT,Cancelled,False,Room-432,2,OPD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,619996,29885,20297,Emergency,2025-01-03 13:02:21.366627,2025-01-03 13:11:21.366627,2025-01-03 13:48:21.366627,2025-01-03 14:09:21.366627,2025-01-03 14:22:21.366627,Completed,False,Room-268,6,ED
119996,619997,11845,20190,New Visit,2025-08-22 11:29:50.759548,2025-08-22 11:16:50.759548,2025-08-22 11:55:50.759548,2025-08-22 12:09:50.759548,2025-08-22 12:29:50.759548,Completed,False,Room-378,4,OPD
119997,619998,34936,20081,Emergency,2024-04-12 11:22:11.041488,2024-04-12 11:21:11.041488,2024-04-12 11:53:11.041488,2024-04-12 12:32:11.041488,2024-04-12 12:44:11.041488,Completed,False,Room-424,8,IPD
119998,619999,14956,20084,Emergency,2024-01-31 03:20:22.356733,2024-01-31 03:29:22.356733,2024-01-31 04:28:22.356733,2024-01-31 05:07:22.356733,2024-01-31 05:20:22.356733,Completed,False,Room-477,4,OPD
